In [4]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BertForSequenceClassification, BertTokenizer
from datasets import load_dataset, load_dataset_builder
from tqdm import tqdm
import numpy as np
import pandas as pd
from huggingface_hub import hf_hub_download
from sklearn.metrics import accuracy_score, f1_score

/media/julia/DATA/Programs/miniconda3/envs/nlp39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# model_handle = 'ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli'
# model_handle = 'chromeNLP/textattack_bert_base_MNLI_fixed'
# model_handle = 'facebook/bart-large-mnli'
# model_name = 'bert-base-uncased-snli-help'
# model_handle = '../models/bert-base-uncased-snli-help/'

dataset_name = 'multi_nli'
split = 'validation_matched'
# split = 'test'

In [6]:
#hf_hub_download(repo_id=model_handle, filename="config.json", cache_dir="../models/ynie")


In [7]:
model = AutoModelForSequenceClassification.from_pretrained(model_handle, resume_download=True)
label2id = model.config.label2id
label2id

Some weights of the model checkpoint at ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'entailment': 0, 'neutral': 1, 'contradiction': 2}

In [9]:
label2id = {
    'bert-base-uncased-snli': {
        'entailment': 1,
        'neutral': 2, 
        'contradiction': 0
    },

    'bert-base-uncased-snli-help': {
        'entailment': 1,
        'neutral': 2, 
        'contradiction': 2
    },
}


In [10]:
builder = load_dataset_builder(dataset_name)
dataset = load_dataset(dataset_name, split=split).filter(lambda x :  x['label']!=-1)
builder.info.features

Found cached dataset multi_nli (/home/julia/.cache/huggingface/datasets/multi_nli/default/0.0.0/591f72eb6263d1ab527561777936b199b714cda156d35716881158a2bd144f39)
Loading cached processed dataset at /home/julia/.cache/huggingface/datasets/multi_nli/default/0.0.0/591f72eb6263d1ab527561777936b199b714cda156d35716881158a2bd144f39/cache-ef0def8b9709d121.arrow


{'promptID': Value(dtype='int32', id=None),
 'pairID': Value(dtype='string', id=None),
 'premise': Value(dtype='string', id=None),
 'premise_binary_parse': Value(dtype='string', id=None),
 'premise_parse': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'hypothesis_binary_parse': Value(dtype='string', id=None),
 'hypothesis_parse': Value(dtype='string', id=None),
 'genre': Value(dtype='string', id=None),
 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None)}

In [11]:
# 
# guess_label2id = {
#     'entailment': 0,
#     'neutral': 2, 
#     'contradiction': 1
# }

# mnli same as dataset label2id
# guess_label2id = {
#     'entailment': 0,
#     'neutral': 1, 
#     'contradiction': 2
# }

# best for roberta-large-mnli
# guess_label2id = {
#     'entailment': 2,
#     'neutral': 1, 
#     'contradiction': 0
# }

#guess for for bert-base-uncased-snli-help
guess_label2id = {
    'entailment': 1,
    'neutral': 2, 
    'contradiction': 2
}

# best for bert-base-uncased-snli
# guess_label2id = {
#     'entailment': 1,
#     'neutral': 2, 
#     'contradiction': 0
# }

dataset = dataset.align_labels_with_mapping(label2id[model_name], 'label')
# dataset = dataset.align_labels_with_mapping(label2id, 'label')
dataset.set_format(type="torch", device='cuda')

NameError: name 'model_name' is not defined

In [ ]:
results = {}
results = pd.DataFrame(results)
results["y_true"] = dataset['label'].to('cpu')
results["y_true"].apply(int)

0       2
1       2
2       1
3       2
4       2
       ..
9810    2
9811    2
9812    1
9813    1
9814    2
Name: y_true, Length: 9815, dtype: int64

In [ ]:
try:
    tokenizer = AutoTokenizer.from_pretrained(model_handle)
except OSError:
    tokenizer = AutoTokenizer.from_pretrained('textattack/bert-base-uncased-snli')
max_length=256
def encode(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], truncation=True, padding="max_length", max_length=max_length)

In [ ]:
dataset = dataset.map(encode, batched=True)
dataset = dataset.map(lambda examples: {"labels": examples["label"]}, batched=True)

dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "premise", "hypothesis",  "labels"], device='cuda')
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64)

Loading cached processed dataset at /home/julia/.cache/huggingface/datasets/multi_nli/default/0.0.0/591f72eb6263d1ab527561777936b199b714cda156d35716881158a2bd144f39/cache-72fe16795de83174.arrow
Loading cached processed dataset at /home/julia/.cache/huggingface/datasets/multi_nli/default/0.0.0/591f72eb6263d1ab527561777936b199b714cda156d35716881158a2bd144f39/cache-ea1d2b65e13ac866.arrow


In [ ]:
df = dataset
df = df[:]
results['premise'] = df['premise']
results['hypothesis'] = df['hypothesis']
results.head()

,y_true,premise,hypothesis
0,2,The new rights are nice enough,Everyone really likes the newest benefits
1,2,This site includes a list of all award winners...,The Government Executive articles housed on th...
2,1,uh i don't know i i have mixed emotions about ...,"I like him for the most part, but would still ..."
3,2,yeah i i think my favorite restaurant is alway...,My favorite restaurants are always at least a ...
4,2,i don't know um do you do a lot of camping,I know exactly.


In [ ]:
model.to('cuda')
model.eval()

with torch.no_grad():
    y_pred = []
    for inputs in tqdm(dataloader):
        batch_outputs = model(inputs['input_ids'], inputs['attention_mask'])
        batch_logits = batch_outputs['logits'].to('cpu')
        batch_predictions = np.argmax(batch_logits, axis=1)
        y_pred += batch_predictions

100%|██████████| 154/154 [01:39<00:00,  1.54it/s]


In [ ]:
results["y_pred"] = y_pred
results["y_pred"] = results["y_pred"].apply(int)

In [ ]:
# results.loc[results.y_true!=results.y_pred].value_counts()
# results.loc[results.y_true==results.y_pred].value_counts()

In [ ]:
#y_pred
#2112
#y_true
#1021

In [12]:
accuracy_score(results["y_true"], results["y_pred"])
# f1_score(results["y_true"], results["y_pred"], average="macro")

NameError: name 'results' is not defined

In [82]:
accuracy_score(results['y_pred'], results['y_true'])

0.5360163015792155

In [83]:
results["y_pred"].loc[results.y_true==0].value_counts()

Series([], Name: y_pred, dtype: int64)

In [63]:
results["y_pred"].loc[results.y_true==1].value_counts()

1    2380
2     965
0     134
Name: y_pred, dtype: int64

In [64]:
results["y_pred"].loc[results.y_true==2].value_counts()

2    2881
1    2627
0     828
Name: y_pred, dtype: int64


| model | accuracy on snli| accuracy on mnli validation_matched|
| --- | --- | --- |
|ynie | 91.8464| --- |
| bert-base-uncased-snli-help | 73.381 | 61.82 |
| chromeNLP | | |